# *1. Setup*

* Import libraries 
* Import data

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
#import libraries

import numpy as np
from collections import Counter
import pandas as pd
import lightgbm as lgbm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import mean_squared_error,roc_auc_score,precision_score,accuracy_score,log_loss
from xgboost import XGBRegressor
import lightgbm as lgb
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_selection import mutual_info_regression

from functools import partial
import optuna

In [ ]:
df_train= pd.read_csv('/kaggle/input/tabular-playground-series-nov-2021/train.csv',index_col=0)
df_test = pd.read_csv('/kaggle/input/tabular-playground-series-nov-2021/test.csv',index_col=0)

# *2. Explore the Data*


In [ ]:
df_train.head()

In [ ]:
df_train.describe()

In [ ]:
ax =sns.countplot(x='target',data=df_train)
ax.set_xticklabels(['Spam','Ham'],ha="right")
plt.title("Spam or Ham")
plt.style.use("seaborn-whitegrid")
total= len(df_train.target)
for p in ax.patches:
    percentage = f'{100 * p.get_height() / total:.1f}%\n'
    x = p.get_x() + p.get_width() / 2
    y = p.get_height()
    ax.annotate(percentage, (x, y), ha='center', va='center')

In [ ]:
#separate targets

y=df_train['target']
features =df_train.drop(['target'],axis=1)


In [ ]:
from sklearn import preprocessing
#scaler = preprocessing.MinMaxScaler()
scaler = preprocessing.StandardScaler()

features[:]= scaler.fit_transform(features[:])

df_test[:] =scaler.transform(df_test[:])

In [ ]:
#split the data to train and test.

X_train,X_test,y_train,y_test = train_test_split(features,y,test_size=0.3,random_state=0)


In [ ]:
#from sklearn.feature_selection import SelectKBest
#from sklearn.feature_selection import chi2
# find best scored 75 features
#select_feature = SelectKBest(chi2, k=75).fit(X_train, y_train)

In [ ]:
#print('Score list:', select_feature.scores_)


In [ ]:
#X_train_2 = pd.DataFrame(select_feature.transform(X_train[:]),columns= range(75))
                         
#X_test_2 = pd.DataFrame(select_feature.transform(X_test[:]),columns=(range(75)))

**Reference**
https://www.kaggle.com/kanncaa1/feature-selection-and-data-visualization
For Feature Selection.

In [ ]:
def objective(trial,X,y, name='xgb'):
    params = param = {
        'tree_method':'gpu_hist',  
        'lambda': trial.suggest_loguniform(
            'lambda', 1e-3, 10.0
        ),
        'alpha': trial.suggest_loguniform(
            'alpha', 1e-3, 10.0
        ),
        'colsample_bytree': trial.suggest_categorical(
            'colsample_bytree', [0.5,0.6,0.7,0.8,0.9,1.0]
        ),
        'subsample': trial.suggest_categorical(
            'subsample', [0.6,0.7,0.8,1.0]
        ),
        'learning_rate': trial.suggest_categorical(
            'learning_rate', [0.008,0.009,0.01,0.012,0.014,0.016,0.018, 0.02]
        ),
        'n_estimators': trial.suggest_categorical(
            "n_estimators", [150, 200, 300, 3000]
        ),
        'max_depth': trial.suggest_categorical(
            'max_depth', [4,5,7,9,11,13,15,17]
        ),
        'random_state': 42,
        'min_child_weight': trial.suggest_int(
            'min_child_weight', 1, 300
        ),
    }

    model =  XGBRegressor(**params)
    model.fit(X_train,y_train,eval_set=[(X_test,y_test)],early_stopping_rounds=50,verbose=False)


    train_score = np.round(np.sqrt(mean_squared_error(y_train, model.predict(X_train))), 5)
    test_score = np.round(np.sqrt(mean_squared_error(y_test, model.predict(X_test))), 5)
                  
    print(f'TRAIN RMSE : {train_score} || TEST RMSE : {test_score}')
                  
    return test_score

In [ ]:
#%%time
#optimize = partial(objective,X=X_train,y=y_train)

#study_lgbm = optuna.create_study(direction ='minimize')
#study_lgbm.optimize(optimize,n_trials=50)


#[I 2021-11-26 19:19:12,114] Trial 29 finished with value: 0.45579 and parameters: {'lambda': 0.03305191955953313, 'alpha': 0.08282886432419537, 'colsample_bytree': 0.8, 'subsample': 0.6, 'learning_rate': 0.008, 'n_estimators': 3000, 'max_depth': 13, 'min_child_weight': 299}. Best is trial 29 with value: 0.45579.
#TRAIN RMSE : 0.39394 || TEST RMSE : 0.45579

In [ ]:
#print(f"\tBest value (rmse): {study_lgbm.best_value:.5f}")
#print(f"\tBest params:")

#for key, value in study_lgbm.best_params.items():
#   print(f"\t\t{key}: {value}")

#Best value (rmse): 0.45579
#	Best params:
#		lambda: 0.03305191955953313
#		alpha: 0.08282886432419537
#		colsample_bytree: 0.8
#		subsample: 0.6
#		learning_rate: 0.008
#		n_estimators: 3000
#		max_depth: 13
#		min_child_weight: 299

In [ ]:
params ={'lambda': 0.03305191955953313,
'alpha': 0.08282886432419537,
'colsample_bytree': 0.8,
'subsample': 0.6,
'learning_rate': 0.008,
'n_estimators': 3000,
'max_depth': 13,
'min_child_weight': 299}

In [ ]:
model =  XGBRegressor(**params,tree_method='gpu_hist')
model.fit(X_train,y_train)

y_pred = model.predict(X_test)

In [ ]:
mean_squared_error(y_test,y_pred,squared=False)

In [ ]:
#df_test_2=pd.DataFrame(select_feature.transform(df_test[:]),columns=range(75))

In [ ]:
#making submission.csv
predictions=model.predict(df_test)
output =pd.DataFrame({'Id': df_test.index,'target':predictions})
output.to_csv('submission.csv',index=False)

**Referecne**

My notebook for tabular-playground-september(I referred many notebooks to use GPU and hyperparameter tuning, and so on.) Needed to predict if people claim the insureance or not.:
https://www.kaggle.com/satoshiss/tabular-playground-september/notebook